Cấu hình và phụ thuộc

Mục đích: thiết lập tham số huấn luyện và thư viện cần dùng.

In [1]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)

from transformers import AutoTokenizer, TFAutoModel
MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

print("Tokenizer OK:", type(tok).__name__)
print("Model OK:", type(bert).__name__)


/Users/lehoangkhang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow: 2.16.2


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Tokenizer OK: BertTokenizerFast
Model OK: TFBertModel


BƯỚC 1 — Đưa đầu vào (Input)

Mục tiêu

Đọc dữ liệu đã tiền xử lý (examples.parquet, vocab_meta.json).

Biến văn bản text thành token ID và attention mask mà BioBERT hiểu (tokenizer của BioBERT).

Tạo tf.data.Dataset cho train và validation (chỉ chuẩn bị đầu vào, chưa xây mô hình).

In [ ]:
# =========================
# BƯỚC 1 — ĐƯA ĐẦU VÀO (INPUT)  [CẬP NHẬT: thêm age & gender]
# =========================

# 1) Import các thư viện cần thiết
import json, random
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import GroupShuffleSplit
from transformers import AutoTokenizer

# 2) Cấu hình cơ bản cho việc tạo batch
MAX_LENGTH = 512   # độ dài tối đa của chuỗi token (cắt nếu dài hơn, pad nếu ngắn hơn)
BATCH_SIZE = 8     # kích thước lô (batch)
VAL_RATIO  = 0.15  # tỉ lệ dành cho validation
SEED = 42          # hạt giống ngẫu nhiên để tái lập kết quả

# 3) Đường dẫn dữ liệu đã tiền xử lý (đã bao gồm gender, age_at_admit)
DATA_ROOT = Path("..") / "data" / "proc"
PARQUET_PATH = DATA_ROOT / "examples.parquet"
VOCAB_PATH   = DATA_ROOT / "vocab_meta.json"

print("Đang dùng:")
print(" - PARQUET_PATH:", PARQUET_PATH.resolve())
print(" - VOCAB_PATH  :", VOCAB_PATH.resolve())

# 4) Kiểm tra sự tồn tại của file dữ liệu
if not PARQUET_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {PARQUET_PATH}. Hãy chạy Notebook tiền xử lý để sinh file, "
        f"hoặc chỉnh lại đường dẫn cho đúng vị trí thực tế."
    )
if not VOCAB_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {VOCAB_PATH}. Hãy kiểm tra Notebook tiền xử lý hoặc đường dẫn."
    )

# 5) Kiểm soát ngẫu nhiên để kết quả ổn định
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 6) Đọc dữ liệu bảng đã tiền xử lý và metadata nhãn
df = pd.read_parquet(PARQUET_PATH)
with open(VOCAB_PATH, "r") as f:
    meta = json.load(f)

icd_vocab        = meta["icd_vocab"]              # danh sách ICD-block (vocab)
lab_vocab_items  = meta["lab_vocab_items"]        # danh sách itemid xét nghiệm (vocab)
proc_vocab = meta["proc_vocab"]     
itemid_to_label  = {int(k): v for k, v in meta["itemid_to_label"].items()}

n_icd = len(icd_vocab)
n_lab = len(lab_vocab_items)
n_proc = len(proc_vocab)

print(f"Số mẫu: {len(df)} | n_icd: {n_icd} | n_proc: {n_proc} | n_lab: {n_lab}")

# 7) Đảm bảo 2 cột nhãn là mảng float32 để dùng với BinaryCrossentropy
df["y_icd"]  = df["y_icd"].apply(lambda a: np.asarray(a, dtype=np.float32))
df["y_proc"] = df["y_proc"].apply(lambda a: np.asarray(a, dtype=np.float32))
df["y_lab"]  = df["y_lab"].apply(lambda a: np.asarray(a, dtype=np.float32))

# 7.1) [MỚI] Chuẩn hoá demographics -> tabular features
# - gender -> one-hot (M/F/U) => 3 chiều
# - age_at_admit -> min-max [0..1] bằng cách chia 120 => 1 chiều
# Tổng số đặc trưng tabular: TAB_DIM = 4
def _normalize_gender(g):
    g = (str(g).upper().strip()[:1] if pd.notna(g) else "U")
    return g if g in ("M", "F", "U") else "U"

df["gender"] = df["gender"].apply(_normalize_gender)
df["age_at_admit"] = df["age_at_admit"].astype("float32")

# one-hot cho gender
gender_to_idx = {"M": 0, "F": 1, "U": 2}
idx = df["gender"].map(gender_to_idx).fillna(2).astype(int).values
gender_onehot = np.eye(3, dtype=np.float32)[idx]   # shape [N, 3]

# age chuẩn hoá (0..120) -> 0..1; thiếu thì 0
age_norm = (df["age_at_admit"].fillna(0.0).clip(0, 120) / 120.0).astype("float32").values.reshape(-1, 1)

# ghép thành vector tabular 4 chiều
tab_feats = np.concatenate([age_norm, gender_onehot], axis=1).astype("float32")  # [N, 4]
TAB_DIM = tab_feats.shape[1]

# 8) Chia train/validation theo subject_id (tránh leakage người bệnh)
groups = df["subject_id"].values
gss = GroupShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=SEED)
train_idx, val_idx = next(gss.split(df, groups=groups))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)

# cắt tương ứng tab_feats
train_tab = tab_feats[train_idx]
val_tab   = tab_feats[val_idx]

print("Số mẫu train:", len(train_df))
print("Số mẫu val  :", len(val_df))
print("TAB_DIM (age_norm + gender_onehot):", TAB_DIM)

# 9) Chuẩn bị tokenizer của BioBERT
try:
    tokenizer = tok  # tái dùng tokenizer đã nạp ở cell trước (nếu có)
except NameError:
    MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 10) Hàm mã hóa 1 lô văn bản sang token ID & attention mask (đầu vào mô hình)
def encode_text_batch(text_list, max_length):
    enc = tokenizer(
        list(text_list),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors=None
    )
    enc["input_ids"] = np.asarray(enc["input_ids"], dtype=np.int32)
    enc["attention_mask"] = np.asarray(enc["attention_mask"], dtype=np.int32)
    return enc

# 11) Tạo tf.data.Dataset theo kiểu generator (thêm 'tab_feats' vào X)
def make_tfds_from_df(frame, tab_array, batch_size, max_length):
    texts = frame["text"].fillna("").tolist()
    y_icd = np.stack(frame["y_icd"].to_list()).astype(np.float32)
    y_proc = np.stack(frame["y_proc"].to_list()).astype(np.float32)   # <<-- mới
    y_lab = np.stack(frame["y_lab"].to_list()).astype(np.float32)

    def gen():
        N = len(texts)
        start = 0
        while start < N:
            end = min(start + batch_size, N)
            enc = encode_text_batch(texts[start:end], max_length=max_length)
            for i in range(end - start):
                yield (
                    {
                        "input_ids": enc["input_ids"][i],
                        "attention_mask": enc["attention_mask"][i],
                        "tab_feats": tab_array[start + i],
                    },
                    {
                        "icd": y_icd[start + i],
                        "proc": y_proc[start + i],    # <<-- mới
                        "lab": y_lab[start + i],
                    }
                )
            start = end

    output_signature = (
        {
            "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            "tab_feats": tf.TensorSpec(shape=(TAB_DIM,), dtype=tf.float32),
        },
        {
            "icd": tf.TensorSpec(shape=(n_icd,), dtype=tf.float32),
            "proc": tf.TensorSpec(shape=(n_proc,), dtype=tf.float32),  # <<-- mới
            "lab": tf.TensorSpec(shape=(n_lab,), dtype=tf.float32),
        },
    )

    ds = tf.data.Dataset.from_generator(gen, output_signature=output_signature)

    ds = ds.padded_batch(
        batch_size,
        padded_shapes=(
            {
                "input_ids": [MAX_LENGTH],
                "attention_mask": [MAX_LENGTH],
                "tab_feats": [TAB_DIM],
            },
            {
                "icd": [n_icd],
                "proc": [n_proc],   # <<-- mới
                "lab": [n_lab],
            },
        ),
        padding_values=(
            {
                "input_ids": tf.constant(0, tf.int32),
                "attention_mask": tf.constant(0, tf.int32),
                "tab_feats": tf.constant(0.0, tf.float32),
            },
            {
                "icd": tf.constant(0.0, tf.float32),
                "proc": tf.constant(0.0, tf.float32),  # <<-- mới
                "lab": tf.constant(0.0, tf.float32),
            },
        ),
        drop_remainder=False
    )
    return ds.prefetch(tf.data.AUTOTUNE)

# 12) Tạo train_ds và val_ds (đã kèm tab_feats)
train_ds = make_tfds_from_df(train_df, train_tab, BATCH_SIZE, MAX_LENGTH).shuffle(1024)
val_ds   = make_tfds_from_df(val_df,   val_tab,   BATCH_SIZE, MAX_LENGTH)

# 13) Kiểm tra nhanh một batch
for batch in train_ds.take(1):
    X, Y = batch
    print("Kích thước đầu vào (X):", {k: v.shape for k, v in X.items()})
    print("Kích thước nhãn (Y):   ", {k: v.shape for k, v in Y.items()})
    # gợi ý: text gốc vẫn xem từ train_df
    print("Ví dụ text gốc (rút gọn):", train_df.iloc[0]["text"][:120].replace("\n", " "))
    break


Đang dùng:
 - PARQUET_PATH: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/examples.parquet
 - VOCAB_PATH  : /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/vocab_meta.json
Số mẫu: 10000 | n_icd: 50 | n_proc: 50 | n_lab: 50
Số mẫu train: 8499
Số mẫu val  : 1501
TAB_DIM (age_norm + gender_onehot): 4
Kích thước đầu vào (X): {'input_ids': TensorShape([8, 256]), 'attention_mask': TensorShape([8, 256]), 'tab_feats': TensorShape([8, 4])}
Kích thước nhãn (Y):    {'icd': TensorShape([8, 50]), 'proc': TensorShape([8, 50]), 'lab': TensorShape([8, 50])}
Ví dụ text gốc (rút gọn): Chief Complaint: altered mental status Major Surgical or Invasive Procedure: ___ Paracentesis ___ ERCP History of Presen


BƯỚC 2 — Xử lý trong BioBERT (tạo vector ngữ nghĩa)

Mục tiêu

Đưa batch input_ids + attention_mask vào BioBERT.

Lấy ra vector biểu diễn câu (dùng token đặc biệt [CLS]).

Kiểm tra kích thước và giá trị điển hình để xác nhận “dịch văn bản → ngôn ngữ số”.

In [4]:
# =========================
# BƯỚC 2 — XỬ LÝ TRONG BIOBERT (CẬP NHẬT)
# =========================

import tensorflow as tf
from transformers import TFAutoModel

MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
DROPOUT = 0.1  # dropout nhẹ cho CLS

# 2.1 Nạp backbone BioBERT (TF). Nếu đã có biến `bert` từ trước thì tái dùng.
try:
    _ = bert  # đã có sẵn
except NameError:
    bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

# 2.2 Inputs của encoder khớp với tf.data: CHỈ gồm input_ids & attention_mask
input_ids      = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# 2.3 Forward qua BioBERT, lấy CLS
bert_outputs = bert(input_ids, attention_mask=attention_mask, training=False)
cls_vec = bert_outputs.last_hidden_state[:, 0, :]  # [B, H]
cls_vec = tf.keras.layers.Dropout(DROPOUT, name="cls_dropout")(cls_vec)

# 2.4 Đóng gói encoder (text-only). Fusion với tab_feats sẽ làm ở bước sau.
encoder = tf.keras.Model(
    inputs={"input_ids": input_ids, "attention_mask": attention_mask},
    outputs={"cls": cls_vec},
    name="biobert_encoder_cls"
)

# 2.5 Kiểm tra nhanh 1 batch
for X_batch, Y_batch in train_ds.take(1):
    # CHỈ truyền 2 khóa mà encoder mong đợi, không đưa 'tab_feats'
    enc_inp = {
        "input_ids": X_batch["input_ids"],
        "attention_mask": X_batch["attention_mask"],
    }
    out = encoder(enc_inp, training=False)
    cls_batch = out["cls"].numpy()
    print("Kích thước vector CLS:", cls_batch.shape)
    print("8 số đầu của CLS[0]:", np.round(cls_batch[0][:8], 4).tolist())
    break


Kích thước vector CLS: (8, 768)
8 số đầu của CLS[0]: [-0.0006000000284984708, -0.07100000232458115, -0.48420000076293945, -0.15230000019073486, -0.32659998536109924, 0.133200004696846, 0.16040000319480896, -0.23240000009536743]


Ý nghĩa của kết quả

Bạn sẽ nhận được ma trận kích thước [BATCH_SIZE, HIDDEN] (thường HIDDEN = 768).

Mỗi hàng là “dấu vân tay số học” của 1 văn bản — chứa ngữ cảnh y khoa đã được BioBERT “đọc hiểu”.

Đây chính là đầu vào cho “phần ra quyết định” ở Bước 3.

BƯỚC 3 — Classifier head (Phần ra quyết định)

Mục tiêu

Gắn 2 “đầu ra quyết định” (Dense) lên vector CLS từ BioBERT:

Đầu ICD: dự đoán đa nhãn các ICD-block.

Đầu Lab: dự đoán đa nhãn các xét nghiệm sớm.

Mỗi đầu trả về logits (số thực âm/dương). Khi qua sigmoid sẽ thành xác suất 0–1 cho từng nhãn.

Kiểm tra kích thước đầu ra và xem thử top-k dự đoán trên một batch (chưa huấn luyện, chỉ để minh họa dòng chảy dữ liệu).

In [6]:
# =========================
# BƯỚC 3 — CLASSIFIER HEAD (CẬP NHẬT: fusion CLS + tab_feats)
# =========================

import tensorflow as tf
import numpy as np

# 3.1 Lấy đầu ra CLS từ encoder (Bước 2)
cls_output = encoder.outputs[0]  # [batch, hidden_size], vd 768

# 3.2 Khai báo thêm input tabular để khớp với tf.data (Bước 1 đã tạo 'tab_feats')
tab_input = tf.keras.Input(shape=(TAB_DIM,), dtype=tf.float32, name="tab_feats")  # [batch, TAB_DIM]

# 3.3 Fusion: ghép CLS + tab_feats (có thể thêm MLP nhỏ để ổn định)
fused = tf.keras.layers.Concatenate(name="fuse_cls_tab")([cls_output, tab_input])  # [batch, 768+TAB_DIM]
fused = tf.keras.layers.Dropout(0.1, name="fuse_dropout")(fused)
fused = tf.keras.layers.Dense(512, activation="relu", name="fuse_dense")(fused)

# 3.4 Ba nhánh logits cho đa nhãn
icd_logits  = tf.keras.layers.Dense(n_icd,  name="icd_logits")(fused)
proc_logits = tf.keras.layers.Dense(n_proc, name="proc_logits")(fused)   # <<-- mới
lab_logits  = tf.keras.layers.Dense(n_lab,  name="lab_logits")(fused)

# 3.5 Mô hình đa nhiệm hoàn chỉnh
multitask_model = tf.keras.Model(
    inputs={
        "input_ids": encoder.inputs[0],      # thay vì encoder.inputs["input_ids"]
        "attention_mask": encoder.inputs[1], # thay vì encoder.inputs["attention_mask"]
        "tab_feats": tab_input,
    },
    outputs={
        "icd": icd_logits,
        "proc": proc_logits,
        "lab": lab_logits,
    },
    name="biobert_multitask_fusion"
)


# 3.6 Kiểm tra kiến trúc
multitask_model.summary(line_length=120)

# 3.7 Chạy thử một batch (X_batch đã có 'input_ids', 'attention_mask', 'tab_feats')
for X_batch, Y_batch in train_ds.take(1):
    logits = multitask_model.predict(X_batch, verbose=0)
    icd_log = logits["icd"]
    lab_log = logits["lab"]
    print("Kích thước icd_logits:", icd_log.shape)
    print("Kích thước lab_logits:", lab_log.shape)

    # Đổi sang xác suất để quan sát
    probs_icd = tf.sigmoid(icd_log).numpy()
    probs_lab = tf.sigmoid(lab_log).numpy()

    k = 5
    top_icd_idx = probs_icd[0].argsort()[-k:][::-1]
    top_lab_idx = probs_lab[0].argsort()[-k:][::-1]
    print("Top-5 ICD indices:", top_icd_idx.tolist())
    print("Top-5 ICD probs  :", np.round(probs_icd[0][top_icd_idx], 4).tolist())

    print("Top-5 Lab indices:", top_lab_idx.tolist())
    print("Top-5 Lab probs  :", np.round(probs_lab[0][top_lab_idx], 4).tolist())

    print("Top-5 ICD codes:", [icd_vocab[i] for i in top_icd_idx])
    print("Top-5 Lab itemids:", [lab_vocab_items[j] for j in top_lab_idx])
    break


Model: "biobert_multitask_fusion"
________________________________________________________________________________________________________________________
 Layer (type)                       Output Shape                        Param #     Connected to                        
 input_ids (InputLayer)             [(None, None)]                      0           []                                  
                                                                                                                        
 attention_mask (InputLayer)        [(None, None)]                      0           []                                  
                                                                                                                        
 tf_bert_model (TFBertModel)        TFBaseModelOutputWithPoolingAndCr   108310272   ['input_ids[0][0]',                 
                                    ossAttentions(last_hidden_state=(                'attention_mask[0][0]']           

Ý nghĩa của kết quả

Bạn sẽ thấy hai ma trận: icd_logits có kích thước [BATCH_SIZE, n_icd], lab_logits có kích thước [BATCH_SIZE, n_lab].

Sau khi áp sigmoid, bạn sẽ nhận được hai ma trận xác suất probs_icd, probs_lab cùng kích thước, mỗi cột là “một ô” xác suất mà bạn mô tả (ví dụ 0.85 cho I21, 0.92 cho Troponin…).

Đây chính là “phần ra quyết định” trước khi so sánh với nhãn thật ở Bước 4.

BƯỚC 4 — So sánh với nhãn thật (Ground truth)

Mục tiêu

Lấy dự đoán từ Bước 3 (logits → xác suất).

So sánh với nhãn thật y_icd, y_lab cho một mẫu trong validation:

Xem những nhãn nào mô hình dự đoán (theo ngưỡng 0.5) so với nhãn thật.

Xem Top-k (ví dụ k=5) nhãn có xác suất cao nhất và đối chiếu nhãn thật.

In [8]:
# ===============================
# BƯỚC 4 — SO SÁNH VỚI NHÃN THẬT (CẬP NHẬT: thêm thủ thuật)
# ===============================

X_batch, Y_batch = next(iter(val_ds))
logits = multitask_model.predict(X_batch, verbose=0)

icd_log  = logits["icd"]
proc_log = logits["proc"]    # <<-- mới
lab_log  = logits["lab"]

probs_icd  = tf.sigmoid(icd_log).numpy()
probs_proc = tf.sigmoid(proc_log).numpy()  # <<-- mới
probs_lab  = tf.sigmoid(lab_log).numpy()

idx = 0
y_icd_true  = Y_batch["icd"][idx].numpy()
y_proc_true = Y_batch["proc"][idx].numpy()  # <<-- mới
y_lab_true  = Y_batch["lab"][idx].numpy()

THR = 0.5
y_icd_pred  = (probs_icd[idx]  >= THR).astype(int)
y_proc_pred = (probs_proc[idx] >= THR).astype(int)  # <<-- mới
y_lab_pred  = (probs_lab[idx]  >= THR).astype(int)

def stats(y_true, y_pred):  
    t, p = np.where(y_true==1)[0], np.where(y_pred==1)[0]
    return len(t), len(p), len(np.intersect1d(t,p))

print("=== So sánh theo NGƯỠNG 0.5 ===")
print("ICD :", *stats(y_icd_true,  y_icd_pred))
print("PROC:", *stats(y_proc_true, y_proc_pred))
print("LAB :", *stats(y_lab_true,  y_lab_pred))

# Có thể thêm so sánh Top-K tương tự nếu cần, chỉ thêm phần PROC theo mẫu ICD/LAB.


=== So sánh theo NGƯỠNG 0.5 ===
ICD : 1 33 0
PROC: 1 30 1
LAB : 2 21 2


Ý nghĩa của kết quả

Bạn thấy rõ cách đối chiếu:

“Dự đoán theo ngưỡng” (≥ 0.5) vs. nhãn thật → đếm đúng/sai.

“Dự đoán Top-k” → xem 5 nhãn cao nhất và đánh dấu nhãn nào trùng nhãn thật.

Đây là bước không tính loss (chưa huấn luyện), chỉ so sánh để hiểu dòng chảy dự đoán ↔ nhãn.
(Loss sẽ thiết lập ở Bước 5.)

BƯỚC 5 — Tính sai số (Loss)

Mục tiêu

Khai báo hàm mất mát cho 2 nhánh đầu ra (ICD và Lab) đúng với thiết kế đa nhãn.

Biên dịch (compile) mô hình với optimizer/metrics, chưa huấn luyện.

Tính thử loss & AUPRC trên 1 batch để thấy mô hình đang ở mức “ngẫu nhiên” trước khi học.

In [10]:
# ======================================
# BƯỚC 5 — TÍNH SAI SỐ (LOSS) & COMPILE (LUÔN TRAIN BIOBERT)
# ======================================

LR = 2e-5

# 1) Loss cho 3 nhánh
losses = {
    "icd":  tf.keras.losses.BinaryCrossentropy(from_logits=True),
    "proc": tf.keras.losses.BinaryCrossentropy(from_logits=True),
    "lab":  tf.keras.losses.BinaryCrossentropy(from_logits=True),
}

# 2) Trọng số
loss_weights = {"icd": 1.0, "proc": 1.0, "lab": 1.0}

# 3) Metrics
metrics = {
    "icd":  [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
    "proc": [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
    "lab":  [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
}

# 4) Optimizer (luôn fine-tune)
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
print("==> TRAIN: Fine-tune toàn bộ BioBERT (không đóng băng)")

# 5) Compile
multitask_model.compile(
    optimizer=optimizer,
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics
)

# 6) Resume checkpoint
OUT_DIR = Path("checkpoints_biobert_multitask")
OUT_DIR.mkdir(parents=True, exist_ok=True)
RESUME_PATH = OUT_DIR / "last"

if RESUME_PATH.exists():
    print(f"==> Resume training từ checkpoint: {RESUME_PATH}")
    multitask_model = tf.keras.models.load_model(RESUME_PATH)
else:
    print("==> Train từ đầu (chưa có checkpoint)")

# 7) Đánh giá nhanh
X_batch, Y_batch = next(iter(val_ds))
results = multitask_model.evaluate(X_batch, Y_batch, verbose=0, return_dict=True)
print("Kết quả trên 1 batch (trước khi train):")
for k, v in results.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, (int, float)) else f"  {k}: {v}")


==> TRAIN: Fine-tune toàn bộ BioBERT (không đóng băng)
==> Train từ đầu (chưa có checkpoint)
Kết quả trên 1 batch (trước khi train):
  loss: 2.2841
  icd_logits_loss: 0.8452
  lab_logits_loss: 0.6880
  proc_logits_loss: 0.7509
  icd_logits_AUPRC: 0.0644
  lab_logits_AUPRC: 0.0632
  proc_logits_AUPRC: 0.0688


BƯỚC 6 — Cập nhật (Training)

Mục tiêu

Huấn luyện toàn bộ mô hình: BioBERT + 2 đầu ra (ICD, Lab) bằng Adam.

Dùng EarlyStopping để dừng sớm nếu không cải thiện.

Lưu checkpoint tốt nhất theo chỉ số thẩm định (validation).

In [ ]:
# ==================================
# BƯỚC 6 — HUẤN LUYỆN (TRAINING) [CẬP NHẬT: hỗ trợ 3 nhánh ICD/PROC/LAB]
# ==================================

import tensorflow as tf
import pandas as pd
from pathlib import Path
import csv, os

OUT_DIR = Path("checkpoints_biobert_multitask")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# === Detect epoch đã train từ log ===
initial_epoch = 0
log_path = OUT_DIR / "train_log.csv"
if log_path.exists():
    try:
        df_log = pd.read_csv(log_path)
        if "epoch" in df_log.columns:
            initial_epoch = int(df_log["epoch"].max()) + 1
    except Exception as e:
        print("⚠️ Không đọc được train_log.csv:", e)

# Train thêm đúng 1 epoch
target_epochs = initial_epoch + 1
print(f"==> Resume training từ epoch {initial_epoch} đến {target_epochs}")


# === Callback custom để log mỗi 1000 batch ===
class BatchLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_file, every_n=1000):
        super().__init__()
        self.log_file = log_file
        self.every_n = every_n
        # nếu file chưa tồn tại thì tạo header
        if not os.path.exists(log_file):
            with open(log_file, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    "epoch", "batch",
                    "loss", "icd_loss", "proc_loss", "lab_loss",
                    "icd_AUPRC", "proc_AUPRC", "lab_AUPRC"
                ])

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = initial_epoch + epoch  # đảm bảo liên tục khi resume

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        if (batch + 1) % self.every_n == 0:
            row = [
                self.epoch, batch + 1,
                logs.get("loss"),
                logs.get("icd_loss"), logs.get("proc_loss"), logs.get("lab_loss"),
                logs.get("icd_AUPRC"), logs.get("proc_AUPRC"), logs.get("lab_AUPRC")
            ]
            # In ra màn hình
            print(f"[Epoch {self.epoch} | Batch {batch+1}] "
                  f"loss={row[2]:.4f}, icd={row[3]:.4f}, proc={row[4]:.4f}, lab={row[5]:.4f}, "
                  f"AUPRC(icd/proc/lab)=({row[6]:.4f}/{row[7]:.4f}/{row[8]:.4f})")
            # Ghi CSV
            with open(self.log_file, "a", newline="") as f:
                csv.writer(f).writerow(row)


batch_logger = BatchLogger(str(OUT_DIR / "batch_log.csv"), every_n=1000)


# === Callbacks khác ===
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_icd_AUPRC",
    mode="max",
    patience=1,
    restore_best_weights=True
)

ckpt_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(OUT_DIR / "best"),
    monitor="val_icd_AUPRC",
    mode="max",
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

ckpt_last = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(OUT_DIR / "last"),
    save_best_only=False,
    save_weights_only=False,
    verbose=0
)

rlrop = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_icd_AUPRC",
    mode="max",
    factor=0.5,
    patience=1,
    min_lr=1e-6,
    verbose=1
)

csv_logger = tf.keras.callbacks.CSVLogger(log_path, append=True)

# === Train thêm 1 epoch ===
history = multitask_model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=initial_epoch,
    epochs=target_epochs,
    callbacks=[early_stop, ckpt_best, ckpt_last, rlrop, csv_logger, batch_logger],
    verbose=0   # tắt log mặc định keras
)

# === Evaluate cuối ===
final_eval = multitask_model.evaluate(val_ds, return_dict=True, verbose=1)
print("\nĐánh giá cuối cùng trên validation:")
for k, v in final_eval.items():
    try:
        print(f"  {k}: {v:.4f}")
    except Exception:
        print(f"  {k}: {v}")

# === Lưu mô hình ===
final_path = OUT_DIR / "final"
multitask_model.save(final_path)
print(f"\nĐã lưu mô hình hiện tại tại: {final_path.resolve()}")
print(f"Checkpoint tốt nhất tại: {(OUT_DIR / 'best').resolve()}")
print(f"Checkpoint mới nhất tại: {(OUT_DIR / 'last').resolve()}")


Ý nghĩa của kết quả

loss, icd_loss, lab_loss trên validation sẽ giảm dần qua epoch; AUPRC sẽ tăng dần nếu dữ liệu/tiền xử lý ổn.

Sau khi huấn luyện, mô hình bắt đầu dự đoán hợp lý hơn (ít “tràn nhãn” ≥ 0.5, top-k sát thực tế hơn).